# Selection of Calibration Windows for Day-Ahead Electricity Price Forecasting
## by Grzegorz Marcjasz, Tomasz Serafin and Rafał Weron 

In [51]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import median_abs_deviation

### Data Preparation


In [117]:
df_day_ahead_epex = pd.read_csv('Day_Ahead_Epex.csv', sep=';')
df_NP2018_complete = pd.read_csv('NP2018_complete.csv')
# remove the first column
df_NP2018_complete = df_NP2018_complete.iloc[:,1:]
df_PJM = pd.read_csv('PJM.csv', sep=';')

In [53]:
def normalize_prices(prices):
    a = np.median(prices)
    b = median_abs_deviation(prices)
    p = 1/b * (prices - a)
    return p, a, b

def apply_vst(p): # apply the area hyperbolic sine transformation to the normalized prices
    X = np.arcsinh(p)
    return X

def inverse_transform(forecast, a, b): # apply the inverse of VST to the forecast to obtain the price prediction
    price_predictions = b * np.sinh(forecast) + a
    return price_predictions

#### df_epex

In [54]:
df_NP2018_complete.head()

,Date,Hour,Price,load,placeholder2,Day
0,20130101,1,31.05,42497.0,2798.0,2
1,20130101,2,30.47,41463.0,2417.0,2
2,20130101,3,28.92,40812.0,2036.0,2
3,20130101,4,27.88,40246.0,1706.0,2
4,20130101,5,26.96,40088.0,1427.0,2


#### df_NP2018

##### noramlization

In [55]:
# Extract 'comed price' column and convert to numpy array
prices = df_NP2018_complete['Price'].values

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Update DataFrame
df_NP2018_complete['Price_norm'] = X

#### df_PJM

In [56]:
df_PJM['sunday'] = np.where(df_PJM['day'] == 7, 1, 0)
df_PJM['saturday'] = np.where(df_PJM['day'] == 6, 1, 0)
df_PJM['monday'] = np.where(df_PJM['day'] ==  1, 1, 0)


##### price normalization

In [57]:
# Extract 'comed price' column and convert to numpy array
prices = df_PJM['comed price'].values

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Update DataFrame
df_PJM['comed_price_norma'] = X

In [65]:
price = df_PJM["comed price"]
exogenos = df_PJM["rto load forecast"]
sunday = df_PJM["sunday"]
saturday = df_PJM["saturday"]
monday = df_PJM["monday"]
df_PJM['Min_Comed_Price'] = df_PJM.groupby('date')['comed price'].transform('min')
df_PJM['Max_Comed_Price'] = df_PJM.groupby('date')['comed price'].transform('max')
df_PJM["min_norm"] = df_PJM.groupby('date')['comed_price_norma'].transform('min')
df_PJM["max_norm"] = df_PJM.groupby('date')['comed_price_norma'].transform('max')

In [66]:
df_PJM.head()

,date,hour,comed price,rto load forecast,comed load foecast,day,sunday,saturday,monday,comed_price_norma,Min_Comed_Price,Max_Comed_Price,for_d_1,min_norm,max_norm
0,20110102,0,17.141179,68590.0,11432.0,1,0,0,1,-1.387638,10.690926,51.389987,NaN,-1.81086,2.083361
1,20110102,1,14.179616,65896.0,10862.0,1,0,0,1,-1.602532,10.690926,51.389987,17.141179,-1.81086,2.083361
2,20110102,2,12.736793,64511.0,10486.0,1,0,0,1,-1.693663,10.690926,51.389987,14.179616,-1.81086,2.083361
3,20110102,3,11.096262,63935.0,10318.0,1,0,0,1,-1.788647,10.690926,51.389987,12.736793,-1.81086,2.083361
4,20110102,4,10.690926,64321.0,10208.0,1,0,0,1,-1.810860,10.690926,51.389987,11.096262,-1.81086,2.083361


## Arx1

#### df_PJM

##### not normalized

In [61]:
load = df_PJM["rto load forecast"]
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed price'].values # hourly data selection
    exogenos = df_PJM.loc[h::24,'rto load forecast'].values
    sunday = df_PJM.loc[h::24,'sunday'].values
    monday = df_PJM.loc[h::24,'monday'].values
    saturday = df_PJM.loc[h::24,'saturday'].values
    c_d_min = df_PJM.loc[h::24,'Min_Comed_Price'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = exogenos[7:T]  #C_d_h
            X6 = sunday[7:T] # sunday
            X7 = monday[7:T] # monday
            X8 = saturday[7:T] # saturday
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))



errors for model 1:  3.095383368363987


In [62]:
# Convert the list of forecasts to a numpy array
forecasts = np.array(forecasts)

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

#### df_NP2018_complete

In [118]:
df_NP2018_complete.head()

,Date,Hour,Price,load,placeholder2,Day
0,20130101,1,31.05,42497.0,2798.0,2
1,20130101,2,30.47,41463.0,2417.0,2
2,20130101,3,28.92,40812.0,2036.0,2
3,20130101,4,27.88,40246.0,1706.0,2
4,20130101,5,26.96,40088.0,1427.0,2


In [119]:
df_NP2018_complete['sunday'] = np.where(df_NP2018_complete['Day'] == 7, 1, 0)
df_NP2018_complete['saturday'] = np.where(df_NP2018_complete['Day'] == 6, 1, 0)
df_NP2018_complete['monday'] = np.where(df_NP2018_complete['Day'] ==  1, 1, 0)

In [110]:
# Extract 'Price' column and convert to numpy array
prices = df_NP2018_complete['Price'].values

# Normalize prices
p, a, b = normalize_prices(prices)

# Apply VST
X = apply_vst(p)

# Update DataFrame
df_NP2018_complete['comed_price_norma'] = X

In [120]:
price = df_NP2018_complete["Price"]
exogenos = df_NP2018_complete["load"]
sunday = df_NP2018_complete["sunday"]
saturday = df_NP2018_complete["saturday"]
monday = df_NP2018_complete["monday"]
df_NP2018_complete['Min_Comed_Price'] = df_NP2018_complete.groupby('Date')['Price'].transform('min')
df_NP2018_complete['Max_Comed_Price'] = df_NP2018_complete.groupby('Date')['Price'].transform('max')

In [121]:
df_NP2018_complete.head()

,Date,Hour,Price,load,placeholder2,Day,sunday,saturday,monday,Min_Comed_Price,Max_Comed_Price
0,20130101,1,31.05,42497.0,2798.0,2,0,0,0,26.96,36.65
1,20130101,2,30.47,41463.0,2417.0,2,0,0,0,26.96,36.65
2,20130101,3,28.92,40812.0,2036.0,2,0,0,0,26.96,36.65
3,20130101,4,27.88,40246.0,1706.0,2,0,0,0,26.96,36.65
4,20130101,5,26.96,40088.0,1427.0,2,0,0,0,26.96,36.65


In [122]:
load = df_NP2018_complete["load"]
errors = []
T = 150
for h in range(24):
    p_hour = df_NP2018_complete.loc[h::24,'Price'].values # hourly data selection
    exogenos = df_NP2018_complete.loc[h::24,'load'].values
    sunday = df_NP2018_complete.loc[h::24,'sunday'].values
    monday = df_NP2018_complete.loc[h::24,'monday'].values
    saturday = df_NP2018_complete.loc[h::24,'saturday'].values
    c_d_min = df_NP2018_complete.loc[h::24,'Min_Comed_Price'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = exogenos[7:T]  #C_d_h
            X6 = sunday[7:T] # sunday
            X7 = monday[7:T] # monday
            X8 = saturday[7:T] # saturday
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))

errors for model 1:  1.8732773354352787


In [116]:
# Convert the list of forecasts to a numpy array
forecasts = np.array(forecasts)

# Apply inverse transformation
price_predictions = inverse_transform(forecasts, a, b)

### Arx2

In [67]:
load = df_PJM["rto load forecast"]
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed_price_norma'].values # hourly data selection
    exogenos = df_PJM.loc[h::24,'rto load forecast'].values
    sunday = df_PJM.loc[h::24,'sunday'].values
    monday = df_PJM.loc[h::24,'monday'].values
    saturday = df_PJM.loc[h::24,'saturday'].values
    c_d_min = df_PJM.loc[h::24,'min_norm'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = exogenos[7:T]  #C_d_h
            X6 = sunday[7:T] # sunday
            X7 = monday[7:T] # monday
            X8 = saturday[7:T] # saturday
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))



errors for model 1:  3.066570716311972


## Arx2

#### df_PJM

##### not normalized

In [68]:
load = df_PJM["rto load forecast"]
df_PJM["for_d_1"] = df_PJM["comed price"].shift(1)
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed price'].values # hourly data selection
    exogenos = df_PJM.loc[h::24,'rto load forecast'].values
    weekday = df_PJM.loc[h::24,'day'].values
    c_d_min = df_PJM.loc[h::24,'Min_Comed_Price'].values
    c_d_max = df_PJM.loc[h::24,'Max_Comed_Price'].values
    for_d_1 = df_PJM.loc[h::24,'for_d_1'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = c_d_max[7:T]  #C_d_h
            X6 = for_d_1[7:T] # c_d-1_24
            X7 = exogenos[7:T]
            X8 = weekday[7:T] 
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))



errors for model 1:  3914.6397079165254


##### normalized

In [70]:
load = df_PJM["rto load forecast"]
df_PJM["for_d_1_norm"] = df_PJM["comed_price_norma"].shift(1)
errors = []
T = 150
for h in range(24):
    p_hour = df_PJM.loc[h::24,'comed_price_norma'].values # hourly data selection
    exogenos = df_PJM.loc[h::24,'rto load forecast'].values
    weekday = df_PJM.loc[h::24,'day'].values
    c_d_min = df_PJM.loc[h::24,'min_norm'].values
    c_d_max = df_PJM.loc[h::24,'max_norm'].values
    for_d_1 = df_PJM.loc[h::24,'for_d_1_norm'].values
    h_errors = []
    # Initialize an empty list to store the forecasts
    forecasts = []
    for day in range(T,len(price)//24):
            cal_data = price[(day-T):day].values # calibration data 
            Y = cal_data[7:T]
            X1 = cal_data[6:T-1] 
            X2 = cal_data[5:T-2] 
            X3 = cal_data[0:T-7]
            X4 = c_d_min[7:T]
            X5 = c_d_max[7:T]  #C_d_h
            X6 = for_d_1[7:T] # c_d-1_24
            X7 = exogenos[7:T]
            X8 = weekday[7:T] 
            X0 = np.ones(np.size(X1)) 
            X = np.stack([X0,X1,X2, X3,X4,X5, X6, X7, X8],axis = 1)
            betas = np.dot(np.linalg.inv(np.dot(X.T,X)), np.dot(X.T,Y))
            X_fut = np.array([1,cal_data[T-1],cal_data[T-2],cal_data[T-7],c_d_min[day], exogenos[day],sunday[day],monday[day],saturday[day]])
            forecast = np.dot(X_fut,betas)
            forecasts.append(forecast)
            real = price[day]
            err = real - forecast
            errors1 = (np.abs(err))
            errors.append(errors1)
print("errors for model 1: " , np.mean(errors))



errors for model 1:  93441.96711814604
